In [ ]:
# pull data from polygon
import datetime
import numpy as np
from scipy.signal import detrend
import requests

def pull_data(ticker, api_key):
    # Define the date range
    end_datetime = datetime.datetime.now()
    start_datetime = end_datetime - datetime.timedelta(days = 365)


    # Polygon API endpoint for aggregate bars
    url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/day/{start_datetime.date()}/{end_datetime.date()}?adjusted=true&apiKey={api_key}"
    
    # Fetch the data
    response = requests.get(url)
    data = response.json()

    if 'results' not in data:
        raise ValueError(f"Error fetching data: {data}")

    # Convert data to arrays
    times = np.array([entry['t'] for entry in data['results']])  # Timestamps
    close = np.array([entry['c'] for entry in data['results']])  # Closing prices

    # Detrend closing prices
    detrended_close = detrend(close)

    return times, close, detrended_close, data

# Example usage:
key = 'jjeryxeZXNkBhTEQF0SDj8uBBI_N1dBM'	
ticker = 'AAPL'
_,close,detrended,df= pull_data(ticker,key)
_,close2,detrended2,df2= pull_data('AMZN',key)


In [ ]:
import numpy as np
from pycwt import wct
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
import scipy.io.wavfile as wav
import fcwt
# Compute the wavelet coherence
def wavelet_coherence(signal1,signal2,highest,lowest,nfreqs,frame_rate):
    freqs, coeffs1 = fcwt.cwt(signal1,frame_rate,lowest,highest,nfreqs,nthreads = 4,)
    freqs, coeffs2 = fcwt.cwt(signal2,frame_rate,lowest,highest,nfreqs,nthreads = 4,)

    # Compute coherence
    S1 = np.abs(coeffs1) ** 2
    S2 = np.abs(coeffs2) ** 2
    S12 = coeffs1 * np.conj(coeffs2)
    
    # Smooth spectra and cross-spectrum
    def smooth(data, sigma=(2,2), mode='nearest'):
        #return data
        return gaussian_filter(data, sigma=sigma, mode=mode)

    S1_smoothed = smooth(S1)
    S2_smoothed = smooth(S2)
    S12_smoothed = smooth(np.abs(S12) ** 2)

    # Coherence calculation
    coherence = S12_smoothed / (((S1_smoothed) **2) * ((S2_smoothed)**2))
    coherence = S12_smoothed / (np.sqrt(S1_smoothed) * np.sqrt(S2_smoothed))
    #coherence = np.clip(coherence, 0.0, 1.0)  # Limit coherence to [0, 1]

    return coherence, freqs, [coeffs1,coeffs2], S12

lowest = 2 # days
highest = 100 # days

coherence, freqs, coeffs, S12 = wavelet_coherence(detrended, detrended2, 1/lowest, 1/highest, 100, 1)

In [ ]:
def coherence_plot_with_arrows(coherence, freqs, S12):
    extent = [0, coherence.shape[1], len(freqs), 0]

    # Normalize coherence for color map
    #coherence = np.clip(coherence, 0, 3)
    norm_coh = (coherence - coherence.min()) / (coherence.max() - coherence.min())

    # Phase angle and vectors
    phase = np.angle(S12)
    U = np.cos(phase)
    V = np.sin(phase)

    # Coordinate grid
    X, Y = np.meshgrid(np.arange(coherence.shape[1]), np.arange(coherence.shape[0]))

    # Mask: only take high-coherence regions
    mask = norm_coh > .3
    indices = np.argwhere(mask)

    # Subsample: take every nth index from the masked points
    subsample_rate = extent[1] // 20  # Adjust this value to control the number of arrows
    subsampled_indices = indices[::subsample_rate]

    x_sub = [X[i, j] for i, j in subsampled_indices]
    y_sub = [Y[i, j] for i, j in subsampled_indices]
    u_sub = [U[i, j] for i, j in subsampled_indices]
    v_sub = [V[i, j] for i, j in subsampled_indices]

    y_values = np.logspace(np.log10(freqs[0]), np.log10(freqs[-1]), len(freqs))

    subsampled_indices = np.linspace(0, len(y_values) - 1, 20, dtype=int)
    subsampled_y_values = y_values[subsampled_indices]
    extent=[0, coherence.shape[1], len(freqs),0]

    # Plot coherence
    fig, ax = plt.subplots(figsize=(10, 6))
    im = ax.imshow(norm_coh, aspect='auto', extent=extent, cmap='Blues')
    ax.quiver(x_sub, y_sub, u_sub, v_sub, color='black', scale=20, headwidth=4
              , headlength=5, headaxislength=4)

    dt = 1 / 250


    # Axes labels and ticks
    ax.set_yticks(subsampled_indices)
    ax.set_yticklabels(subsampled_y_values.round(2))
    ax.set_xticks(np.linspace(0, coherence.shape[1], 5))
    ax.set_ylabel('Frequency', fontsize=14)
    ax.set_xlabel('Time', fontsize=14)
    #ax.set_title('Wavelet Coherence', fontsize=16)
    fig.colorbar(im, ax=ax)
    #ax.legend()

    return fig



In [ ]:
from multiprocessing import Process, Queue
import numpy as np
import tracemalloc
import pycwt

mother = pycwt.Morlet(6)

# Dummy test data
sampling_rate = 8000
nfreqs = 100
t = np.arange(0, 1, 1 / sampling_rate)
noise1 =np.random.randn(len(t))
noise2 =np.random.randn(len(t))
def compute_wavelet_params(f_min, f_max, nscales, dt, f_c=0.8125):
    a_min = f_c / f_max
    a_max = f_c / f_min
    dj = np.log2(a_max / a_min) / nscales
    s0 = a_min * dt
    J = nscales - 1
    return dj, s0, J

def run_coherence_a(q):
    tracemalloc.start()
    coh, _ = wavelet_coherence(noise1, noise2, 4000, 200, 100, sampling_rate)
    _, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    q.put(f"Coherence A: Peak RAM = {peak / 1024 / 1024:.2f} MB")

def run_coherence_b(q):

    dj, s0, J = compute_wavelet_params(200, 4000, nfreqs, 1 / sampling_rate)

    tracemalloc.start()
    coh2, *_ = pycwt.wct(noise1, noise2, 1 / sampling_rate, dj=dj, s0=s0, J=J, wavelet=mother, sig=False)
    _, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    q.put(f"Coherence B: Peak RAM = {peak / 1024 / 1024:.2f} MB")


p1 = Process(target=run_coherence_a, )
p2 = Process(target=run_coherence_b,)

p1.start()
p1.join()
p2.start()
p2.join()


In [ ]:


# --- WAVELET COHERENCE FUNCTION ---
def wavelet_coherence(signal1, signal2, highest_freq, lowest_freq, nfreqs, sampling_rate):
    freqs, coeffs1 = fcwt.cwt(signal1, sampling_rate, lowest_freq, highest_freq, nfreqs, nthreads=4,scaling='log')
    _, coeffs2 = fcwt.cwt(signal2, sampling_rate, lowest_freq, highest_freq, nfreqs, nthreads=4,scaling='log')
    
    S1 = np.abs(coeffs1) ** 2
    S2 = np.abs(coeffs2) ** 2
    S12 = coeffs1 * np.conj(coeffs2)

    def smooth(data, sigma=(2, 2), mode='nearest'):
        return gaussian_filter(data, sigma=sigma, mode=mode)

    S1_smooth = smooth(S1)
    S2_smooth = smooth(S2)
    S12_smooth = smooth(np.abs(S12) ** 2)

    coherence = S12_smooth / (np.sqrt(S1_smooth) * np.sqrt(S2_smooth))
    return coherence, freqs

def coherence(coeffs1, coeffs2, freqs):
    S1 = np.abs(coeffs1) ** 2
    S2 = np.abs(coeffs2) ** 2
    S12 = coeffs1 * np.conj(coeffs2)

    def smooth(data, sigma=(2, 2), mode='nearest'):
        return gaussian_filter(data, sigma=sigma, mode=mode)

    S1_smooth = smooth(S1)
    S2_smooth = smooth(S2)
    S12_smooth = smooth(np.abs(S12) ** 2)

    coh = S12_smooth / (np.sqrt(S1_smooth) * np.sqrt(S2_smooth))
    return coh, freqs, S12


In [ ]:
def transform(signal1,frame_rate,highest,lowest):
    nfreqs = 100
    freqs, coeffs1 = fcwt.cwt(signal1,frame_rate,lowest,highest,nfreqs,nthreads = 4,scaling='log')

    return coeffs1,freqs

def transform_plot(signal,frame_rate,highest,lowest):


    coeffs1,freqs = transform(signal,frame_rate,highest,lowest)

    y_values = np.logspace(np.log10(freqs[0]), np.log10(freqs[-1]), len(freqs))
    subsampled_indices = np.linspace(0, len(y_values) - 1, 20, dtype=int)
    subsampled_y_values = y_values[subsampled_indices]
    extent=[0, len(signal)/frame_rate, len(freqs),0]
    plt.figure(figsize=(10, 5));
    plt.imshow(np.abs(coeffs1), aspect='auto', extent=extent, cmap='jet')
    plt.yticks(ticks=subsampled_indices, labels=[f"{y:.1f}" for y in subsampled_y_values])

    cbar = plt.colorbar()
    cbar.ax.set_ylabel('Magnitude')
    cbar.set_ticks([])  
    plt.xlabel('Time (Years)')
    plt.ylabel('Frequency (per year)')
    #plt.title('Wavelet Transform Magnitude')
    plt.tight_layout()
    plt.show()
    return coeffs1,freqs

#transform_plot(detrended2,250,125,3)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
fs = 1000  # Sampling rate
duration = 6  # seconds
t = np.arange(0, duration, 1/fs)

# Frequencies
f1 = 10    # Hz
f2 = 75    # Hz

# Phase lag: 1/4 cycle = π/2 radians
phase_shift = np.pi / 2

# Time-localized windows
mask_10hzx = (t >= 1) & (t <= 3.0)
mask_75hzx = (t >= 0.2) & (t <= 4.4)

mask_10hzy = (t >= 1.2) & (t <= 3.2)
mask_75hzy = (t >= 0.4) & (t <= 5.6)

# Signal X (no phase shift)
x = np.zeros_like(t)
x[mask_10hzx] += np.sin(2 * np.pi * f1 * t[mask_10hzx])
x[mask_75hzx] += np.sin(2 * np.pi * f2 * t[mask_75hzx])

# Signal Y (π/2 phase shift)
y = np.zeros_like(t)
y[mask_10hzy] += np.sin(2 * np.pi * f1 * t[mask_10hzy] + phase_shift)
y[mask_75hzy] += np.sin(2 * np.pi * f2 * t[mask_75hzy] + phase_shift)

# Add white Gaussian noise
noise_level = 0.2
x += np.random.normal(0, noise_level, size=t.shape)
y += np.random.normal(0, noise_level, size=t.shape)
fig,ax = plt.subplots(2,1,figsize=(12,6))
# Plot
ax[0].plot(t, x, label='Signal X', alpha=0.7)
ax[0].set_xlim(0, 6)
ax[0].set_xlabel("Time (s)")
ax[0].set_ylabel("Amplitude")
#ax[0].set_title("Signal X")
ax[0].legend()
ax[0].grid(True)
ax[1].plot(t, y, label='Signal Y', alpha=0.7)
ax[1].set_xlim(0, 6)
ax[1].set_xlabel("Time (s)")
ax[1].set_ylabel("Amplitude")
#ax[1].set_title("Signal Y")
ax[1].legend()
ax[1].grid(True)



In [ ]:
def coherence_plot(coh,freqs,sampling_rate):
    y_values = np.logspace(np.log10(freqs[0]), np.log10(freqs[-1]), len(freqs))

    subsampled_indices = np.linspace(0, len(y_values) - 1, 20, dtype=int)
    subsampled_y_values = y_values[subsampled_indices]
    extent=[0, coh.shape[1]/sampling_rate, len(freqs),0]
    plt.figure(figsize=(10, 5));
    plt.imshow(coh, aspect='auto', extent=extent, cmap='Blues')
    plt.yticks(ticks=subsampled_indices, labels=[f"{y:.1f}" for y in subsampled_y_values])

    plt.colorbar(label='Coherence')
    plt.xlabel('Time (years)')
    plt.ylabel('Frequency (per year)')
    plt.title('Wavelet Coherence')
    plt.tight_layout()
    plt.show()

def workflow(sig1,sig2,bounds,phase = True):
    sampling_rate = 250 # Assuming days per year
    highest = bounds[0]
    lowest = bounds[1]
    t1,_ = transform(sig1, 250, highest, lowest)
    t2,freqs = transform(sig2, 250, highest, lowest)
    coh, _, S12 = coherence(t1, t2, freqs)
    if phase:
        coherence_plot_with_arrows(coh, freqs, S12)
    else:
        coherence_plot(coh, freqs, 250)
    return freqs,S12,coh


workflow()

In [54]:
import numpy as np

def generate_nonstationary_signal(duration, sampling_rate, freq_range):
    t = np.linspace(0, duration, int(sampling_rate * duration), endpoint=False)
    signal = np.zeros_like(t)

    # Embed frequencies within the specified range
    for freq in np.linspace(freq_range[0], freq_range[1], num=5):  # 5 frequencies
        amplitude = np.random.uniform(0.5, 1.5)  # Random amplitude
        phase = np.random.uniform(0, 2 * np.pi)  # Random phase
        modulation = np.exp(-0.5 * ((t - duration / 2) / (duration / 4))**2)  # Gaussian envelope
        signal += amplitude * modulation * np.sin(2 * np.pi * freq * t + phase)

    # Add random noise
    signal += 0.2 * np.random.randn(len(t))
    return t, signal

# Parameters
duration = 10  # seconds
sampling_rate = 250  # Hz
freq_range = (10, 100)  # Hz

In [ ]:
# filename: wavelet_dashboard.ipynb

import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

# Your functions: transform, coherence, coherence_plot, coherence_plot_with_arrows, pull_data

def workflow(sig1, sig2, bounds, phase=True):
    sampling_rate = 250  # days/year
    highest, lowest = bounds
    t1, _ = transform(sig1, sampling_rate, highest, lowest)
    t2, freqs = transform(sig2, sampling_rate, highest, lowest)
    coh, _, S12 = coherence(t1, t2, freqs)
    
    #fig = plt.figure(figsize=(10, 6));
    if phase:
        coherence_plot_with_arrows(coh, freqs, S12)
    else:
        coherence_plot(coh, freqs, sampling_rate)
    plt.show()

    return freqs, S12, coh

# --- Widgets ---
ticker1 = widgets.Text( description='Stock 1:')
ticker2 = widgets.Text( description='Stock 2:')
upper_bound = widgets.IntSlider(value=125, min=20, max=250, step=5, description='Max Period:')
lower_bound = widgets.IntSlider(value=8, min=2, max=50, step=1, description='Min Period:')
phase_toggle = widgets.Checkbox(value=True, description='Show Phase Arrows')
run_button = widgets.Button(description='Run Analysis')

import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

output = widgets.Output()

def on_run_clicked(b):
    with output:
        clear_output(wait=True)
        sig1 = pull_data(ticker1.value, key)[2]
        sig2 = pull_data(ticker2.value, key)[2]
        bounds = (upper_bound.value, lower_bound.value)
        workflow(sig1, sig2, bounds, phase=phase_toggle.value)

run_button.on_click(on_run_clicked)

# Layout
ui = widgets.VBox([
    widgets.HBox([ticker1, ticker2]),
    upper_bound,
    lower_bound,
    phase_toggle,
    run_button,
    output
])

display(ui)
